In [1]:
!pip install transformers datasets
!pip install transformers
!pip install transformers[torch]
!pip install transformers[tf-cpu]
!pip install transformers[flax]
!pip install torch
!pip install tensorflow
!pip install evaluate
!pip install torchsummary
!pip install gdown

INFO: pip is looking at multiple versions of transformers[tf-cpu] to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement tensorflow-cpu<2.13,>=2.4; extra == "tf-cpu" (from transformers[tf-cpu]) (from versions: none)
ERROR: No matching distribution found for tensorflow-cpu<2.13,>=2.4; extra == "tf-cpu"
INFO: pip is looking at multiple versions of transformers[flax] to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement jax!=0.3.2,<=0.3.6,>=0.2.8; extra == "flax" (from transformers[flax]) (from versions: none)
ERROR: No matching distribution found for jax!=0.3.2,<=0.3.6,>=0.2.8; extra == "flax"
ERROR: Could not find a version that satisfies the requirement evaluate (from versions: none)
ERROR: No matching distribution found for evaluate
ERROR: Could not find a version that satisfies the requir

In [16]:
import multiprocessing
import pandas as pd
import numpy as npp
import torch
import matplotlib.pyplot as plt
import transformers
import torch.nn as nn
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoModelForMaskedLM, AutoModelForPreTraining, EarlyStoppingCallback, AutoModelForSequenceClassification, BertForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from transformers import BertForMaskedLM, DistilBertForMaskedLM
from transformers import BertTokenizer, DistilBertTokenizer
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from tokenizers import BertWordPieceTokenizer

In [ ]:
# torch.set_default_device('cuda')

In [ ]:
# import gdown

# dataset_link = "https://drive.google.com/uc?id=1FoyySUxspmkT4KNwSsYWDqw8TVUF-meV"

# gdown.download(dataset_link,"checkpoint.zip")



In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoConfig

model_name = "csebuetnlp/banglabert"

tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained("/kaggle/input/vitd-dataset-and-models/last training checkpoint for finetuning/last training checkpoint for finetuning/config.json")

model = AutoModel.from_pretrained("/kaggle/input/vitd-dataset-and-models/last training checkpoint for finetuning/last training checkpoint for finetuning/pytorch_model.bin",config=config)

classifier = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=3)

# classifier.encoder = model.encoder
# classifier.embeddings = model.embeddings
# model

classifier.electra.embeddings = model.embeddings
classifier.electra.encoder = model.encoder
# for i in range(7):
#     classifier.electra.encoder.layer[i] = model.encoder.layer[i]
classifier


In [ ]:
# for param in classifier.electra.embeddings.parameters():
#   param.requires_grad=False

for i in range(3):
  for param in classifier.electra.encoder.layer[i].parameters():
    param.requires_grad=False

# for param in classifier.electra.encoder.parameters():
#   param.requires_grad=False

# classifier.electra.encoder.layer[0]

In [ ]:
for name, param in classifier.named_parameters():
  print("name: ",name," ","Trainable: ",param.requires_grad)

In [ ]:
# import pandas as pd
train_df = pd.read_csv("/kaggle/input/vitd-datasets/trainupscaledbycopy.csv")
val_df = pd.read_csv("/kaggle/input/vitd-datasets/dev.csv")

In [ ]:
# train_df.head()

In [ ]:
from sklearn.model_selection import train_test_split
def pandaToList(dataset):
    text_list = []
    label_list = []
    for i in range(len(dataset['text'])):
      text_list.append(dataset['text'][i])
      label_list.append(int(dataset['label'][i]))
    return text_list, label_list

train_texts, train_labels =  pandaToList(train_df)
val_texts, val_labels = pandaToList(val_df)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
# print(train_encodings['input_ids'][0])
# print(val_encodings["input_ids"][0])

In [ ]:
import torch

class ViolenceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ViolenceDataset(train_encodings, train_labels,)
val_dataset = ViolenceDataset(val_encodings, val_labels)

In [ ]:


from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}



In [ ]:
from transformers import  Trainer, TrainingArguments, EarlyStoppingCallback, IntervalStrategy, AutoModelForSequenceClassification

training_args = TrainingArguments(
   f"training_with_callbacks",
   evaluation_strategy = IntervalStrategy.STEPS, # "steps"
   #output_dir= output_dir,
  #  output_dir="/content/drive/MyDrive/model_emnlp/buetcsenlp_banglabert_finetuned_classifier",
   eval_steps = 250, # Evaluation and Save happens every 250 steps
   save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.

   learning_rate=2e-5,
   per_device_train_batch_size=1,
   per_device_eval_batch_size=1,
   num_train_epochs=50,
   weight_decay=0.01,
   metric_for_best_model = 'f1',
   report_to= "wandb",
  #  generator=torch.Generator(device='cuda'),
   load_best_model_at_end=True)

trainer = Trainer(
    model=classifier,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    # generator=torch.Generator(device='cuda'),
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)
# trainer.to('cuda')

In [ ]:
trainer.train()

In [ ]:
!pip install evaluate

In [ ]:
test_df = pd.read_csv("/kaggle/input/vitd-datasets/test(2).csv")
test_texts, test_labels =  pandaToList(test_df)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
test_dataset = ViolenceDataset(train_encodings, train_labels,)

In [ ]:
from transformers import TextClassificationPipeline

pipeline = TextClassificationPipeline(model=classifier,tokenizer=tokenizer,return_all_scores=True)
output = pipeline(test_df["text"].tolist())
# print(output)
